In [ ]:
!pip install opencv-python
!pip install np
!pip install matplotlib
!pip install tqdm
!pip install image
!pip install natsort

  Created wheel for np: filename=np-1.0.2-py3-none-any.whl size=13676 sha256=edf804fdf2ae1e1d24de941735565db6d79757d21568dd863ca46118f59f95a5
  Stored in directory: /root/.cache/pip/wheels/8d/31/5b/f3f27c678f2b3ad7e29903ed09bb7446717fd4c8b35f53973a
Successfully built np
     |████████████████████████████████| 7.9 MB 5.3 MB/s 
  Created wheel for image: filename=image-1.5.33-py2.py3-none-any.whl size=19496 sha256=1bffe7f5ea1aba37c0717187fdb1918539ccbce749bf2367f190e5418cd601e4
  Stored in directory: /root/.cache/pip/wheels/56/88/e6/897194cfe8c08a8b9afd881d3bf53d102e13fa39607d721383
Successfully built image


In [ ]:
!pip3 install firebase-admin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### 모듈 import

In [ ]:
import os
import pickle
import cv2
import numpy as np
import natsort
from matplotlib import pyplot as plt
from tqdm import tqdm
from PIL import Image

### 경로 설정

In [ ]:
images_dir_path = os.path.join(os.getcwd(), '/content/C:/Users/LeeYuho/Desktop/video/test')
videofilepath = '/content/C:/Users/LeeYuho/Desktop/video/test.mov'

### 비디오 정보 출력

In [ ]:
video = cv2.VideoCapture(videofilepath)
if not video.isOpened():
    print("Could not Open :", videofilepath)
    exit(0)

#불러온 비디오 파일의 정보 출력
length = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = video.get(cv2.CAP_PROP_FPS)

print("length :", length)
print("width :", width)
print("height :", height)
print("fps :", fps)    

try:
    if not os.path.exists(videofilepath[:-4]):
        os.makedirs(videofilepath[:-4])
except OSError:
    print ('Error: Creating directory. ' +  videofilepath[:-4])


length : 332
width : 568
height : 320
fps : 29.97742663656885


### 비디오 프레임 이미지 분할

In [ ]:
count = 0
while(video.isOpened()):
    ret, image = video.read()
    if(int(video.get(1)) % 30 == 0): #앞서 불러온 fps 값을 사용하여 1초마다 추출
        cv2.imwrite(videofilepath[:-4] + "/frame%d.jpg" % count, image)
        print('Saved frame number :', str(int(video.get(1))))
        count += 1

Saved frame number : 30
Saved frame number : 60
Saved frame number : 90
Saved frame number : 120
Saved frame number : 150
Saved frame number : 180
Saved frame number : 210
Saved frame number : 240
Saved frame number : 270
Saved frame number : 300
Saved frame number : 330


KeyboardInterrupt: ignored

### 히스토그램 그리기

In [ ]:
def draw_color_histogram_from_image(file_name, image):
  print(image)
  # Image 시각화
  f=plt.figure(figsize=(10,3))
  im1 = f.add_subplot(1,2,1)
  im1.grid(False)
  im1.imshow(image)
  im1.set_title(filename)

  # Histogram 시각화
  im2 = f.add_subplot(1,2,2)

  histr = cv2.calcHist([image],[0],None,[256],[0,256])
  im2.plot(histr,color = 'k')

  im2.grid(True)
  im2.set_title(file_name + "Histogram")

### 히스토그램 추출

In [ ]:
def get_histogram(image):
    histogram = []

    # Create histograms per channels, in 4 bins each.
    
    channel_histogram = cv2.calcHist(images=[image],
                                     channels=[0],
                                     mask=None,
                                     histSize=[4],  # 히스토그램 구간을 4개로 한다.
                                     ranges=[0, 256])
    histogram.append(channel_histogram)

    histogram = np.concatenate(histogram)
    histogram = cv2.normalize(histogram, histogram)

    return histogram

### 파일 불러오기 및 정렬

In [ ]:
filenameList = natsort.natsorted(os.listdir(images_dir_path))
print(filenameList)

['frame0.jpg', 'frame1.jpg', 'frame2.jpg', 'frame3.jpg', 'frame4.jpg', 'frame5.jpg', 'frame6.jpg', 'frame7.jpg', 'frame8.jpg', 'frame9.jpg', 'frame10.jpg']


### 이미지 크롭

In [ ]:
def image_crop(image):
  WIDTH_SCALE = 4
  HEIGHT_SCALE = 4
  width, height = image.shape
  w = width // WIDTH_SCALE
  h = height // HEIGHT_SCALE
  
  image_crop_list = []
  v_crop_list = np.vsplit(image, HEIGHT_SCALE)
  for v_crop in v_crop_list:
    h_crop_list = np.hsplit(v_crop, WIDTH_SCALE)
    for crop in h_crop_list:
      image_crop_list.append(crop) 
    
  return image_crop_list

### 히스토그램 연산 실행

In [ ]:
WIDTH_SCALE = 4
HEIGHT_SCALE = 4
prev_img_str = None
prev_histogram_list = []
for filename in filenameList:
  file_path = os.path.join(images_dir_path, filename)
  image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
  crop_list = image_crop(image)
  histogram_list = []
  distance = [0] * (WIDTH_SCALE * HEIGHT_SCALE)
  for crop in crop_list:
    crop = np.array(crop)
    histogram = get_histogram(crop)
    histogram_list.append(histogram)
    curr_img_str = filename
    # draw_color_histogram_from_image(filename, crop)
  if len(prev_histogram_list) != 0:
    for i in range(WIDTH_SCALE*HEIGHT_SCALE):
      distance[i] = cv2.compareHist(H1=prev_histogram_list[i],
                                    H2=histogram_list[i],
                                    method=cv2.HISTCMP_CHISQR)
  prev_histogram_list = histogram_list[:]
  for i in range(WIDTH_SCALE*HEIGHT_SCALE):
    if i == 0: print("prev <-> " + filename)
    if i%4 == 0: print()
    print("%.3f"%distance[i],end='  ')
  print("\n-----------------------")


  

prev <-> frame0.jpg

0.000  0.000  0.000  0.000  
0.000  0.000  0.000  0.000  
0.000  0.000  0.000  0.000  
0.000  0.000  0.000  0.000  
-----------------------
prev <-> frame1.jpg

0.190  0.012  0.002  0.001  
0.102  0.009  0.009  0.011  
0.002  0.043  0.035  0.030  
0.027  0.004  0.006  0.012  
-----------------------
prev <-> frame2.jpg

0.037  0.009  0.002  0.014  
0.043  0.007  0.005  0.008  
0.000  0.041  0.017  0.001  
0.014  0.000  0.001  0.015  
-----------------------
prev <-> frame3.jpg

0.079  0.017  0.011  0.013  
0.023  0.006  0.044  0.203  
0.005  0.043  0.029  0.012  
0.021  0.006  0.008  0.012  
-----------------------
prev <-> frame4.jpg

0.020  0.028  0.004  0.001  
0.017  0.001  0.003  0.072  
0.003  0.062  0.024  0.067  
0.105  0.035  0.022  0.029  
-----------------------
prev <-> frame5.jpg

0.137  77.537  0.371  0.016  
0.047  0.017  0.102  0.024  
0.013  0.021  0.013  0.012  
0.010  0.011  0.017  0.007  
-----------------------
prev <-> frame6.jpg

0.002  0.800

In [ ]:
import os
import pickle
import cv2
import numpy as np
import natsort
from matplotlib import pyplot as plt
from tqdm import tqdm
from PIL import Image


images_dir_path = os.path.join(os.getcwd(), '/content/C:/Users/LeeYuho/Desktop/video/test')
videofilepath = '/content/C:/Users/LeeYuho/Desktop/video/test.mov'

video = cv2.VideoCapture(videofilepath)
if not video.isOpened():
    print("Could not Open :", videofilepath)
    exit(0)

#불러온 비디오 파일의 정보 출력
length = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = video.get(cv2.CAP_PROP_FPS)

print("length :", length)
print("width :", width)
print("height :", height)
print("fps :", fps)    

try:
    if not os.path.exists(videofilepath[:-4]):
        os.makedirs(videofilepath[:-4])
except OSError:
    print ('Error: Creating directory. ' +  videofilepath[:-4])

def draw_color_histogram_from_image(file_name, image):
  print(image)
  # Image 시각화
  f=plt.figure(figsize=(10,3))
  im1 = f.add_subplot(1,2,1)
  im1.grid(False)
  im1.imshow(image)
  im1.set_title(filename)

  # Histogram 시각화
  im2 = f.add_subplot(1,2,2)

  histr = cv2.calcHist([image],[0],None,[256],[0,256])
  im2.plot(histr,color = 'k')

  im2.grid(True)
  im2.set_title(file_name + "Histogram")

def get_histogram(image):
    histogram = []

    # Create histograms per channels, in 4 bins each.
    
    channel_histogram = cv2.calcHist(images=[image],
                                     channels=[0],
                                     mask=None,
                                     histSize=[4],  # 히스토그램 구간을 4개로 한다.
                                     ranges=[0, 256])
    histogram.append(channel_histogram)

    histogram = np.concatenate(histogram)
    histogram = cv2.normalize(histogram, histogram)

    return histogram

def image_crop(image):
  WIDTH_SCALE = 4
  HEIGHT_SCALE = 4
  width, height = image.shape
  w = width // WIDTH_SCALE
  h = height // HEIGHT_SCALE
  
  image_crop_list = []
  v_crop_list = np.vsplit(image, HEIGHT_SCALE)
  for v_crop in v_crop_list:
    h_crop_list = np.hsplit(v_crop, WIDTH_SCALE)
    for crop in h_crop_list:
      image_crop_list.append(crop) 
    
  return image_crop_list

filenameList = natsort.natsorted(os.listdir(images_dir_path))
WIDTH_SCALE = 4
HEIGHT_SCALE = 4
prev_img_str = None
prev_histogram_list = []
for filename in filenameList:
  file_path = os.path.join(images_dir_path, filename)
  image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
  crop_list = image_crop(image)
  histogram_list = []
  distance = [0] * (WIDTH_SCALE * HEIGHT_SCALE)
  for crop in crop_list:
    crop = np.array(crop)
    histogram = get_histogram(crop)
    histogram_list.append(histogram)
    curr_img_str = filename
    # draw_color_histogram_from_image(filename, crop)
  if len(prev_histogram_list) != 0:
    for i in range(WIDTH_SCALE*HEIGHT_SCALE):
      distance[i] = cv2.compareHist(H1=prev_histogram_list[i],
                                    H2=histogram_list[i],
                                    method=cv2.HISTCMP_CHISQR)
  prev_histogram_list = histogram_list[:]
  for i in range(WIDTH_SCALE*HEIGHT_SCALE):
    if i == 0: print("prev <-> " + filename)
    if i%4 == 0: print()
    print("%.3f"%distance[i],end='  ')
  print("\n-----------------------")


  

length : 332
width : 568
height : 320
fps : 29.97742663656885
prev <-> frame0.jpg

0.000  0.000  0.000  0.000  
0.000  0.000  0.000  0.000  
0.000  0.000  0.000  0.000  
0.000  0.000  0.000  0.000  
-----------------------
prev <-> frame1.jpg

0.190  0.012  0.002  0.001  
0.102  0.009  0.009  0.011  
0.002  0.043  0.035  0.030  
0.027  0.004  0.006  0.012  
-----------------------
prev <-> frame2.jpg

0.037  0.009  0.002  0.014  
0.043  0.007  0.005  0.008  
0.000  0.041  0.017  0.001  
0.014  0.000  0.001  0.015  
-----------------------
prev <-> frame3.jpg

0.079  0.017  0.011  0.013  
0.023  0.006  0.044  0.203  
0.005  0.043  0.029  0.012  
0.021  0.006  0.008  0.012  
-----------------------
prev <-> frame4.jpg

0.020  0.028  0.004  0.001  
0.017  0.001  0.003  0.072  
0.003  0.062  0.024  0.067  
0.105  0.035  0.022  0.029  
-----------------------
prev <-> frame5.jpg

0.137  77.537  0.371  0.016  
0.047  0.017  0.102  0.024  
0.013  0.021  0.013  0.012  
0.010  0.011  0.017  0.0

In [ ]:
from time import sleep
from picamera import PiCamera
import datetime
import sys, os
import requests
import firebase_admin
from firebase_admin import credentials
from firebase_admin import storage
from uuid import uuid4
import schedule

PROJECT_ID = "tennisapp-be86b"#자신의 project id

if not firebase_admin._apps:
  cred = credentials.Certificate("/content/ServiceAccountKey.json")
  default_app = firebase_admin.initialize_app(cred, {
      #gs://smart-mirror-cf119.appspot.com
      'storageBucket': f"{PROJECT_ID}.appspot.com"    
  })
  #버킷은 바이너리 객체의 상위 컨테이너이다. 버킷은 Storage에서 데이터를 보관하는 기본 컨테이너이다.
  bucket = storage.bucket()#기본 버킷 사용

def fileUpload(file):
    blob = bucket.blob('/content/images/'+file)
    #new token and metadata 설정
    new_token = str(uuid4())
    metadata = {"firebaseStorageDownloadTokens": new_token} #access token이 필요하다.
    blob.metadata = metadata

    #upload file
    blob.upload_from_filename(filename='/content/images/'+file, content_type='image/jpeg')
    print(blob.public_url)

def execute_camera():
    
    #사진찍기
    #중복없는 파일명 만들기
    basename = "smr"
    suffix = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + '.jpg'
    filename = "_".join([basename, suffix])

    camera = PiCamera()
    camera.resolution = (640, 480)
    camera.start_preview()
    #이미지에 텍스트를 새겨 넣자.
    camera.annotate_text = "Smart Mirror"
    camera.annotate_text_size = 20
    sleep(5)
    #사진을 찍어서 저장한다. 파일의 중복되지 않도록 날짜시간을 넣어서 만듬
    camera.capture('/home/pi/python/captureImages/' + filename)
    #사진 파일을 파이어베이스에 업로드 한다.
    fileUpload(filename)
    #로컬 하드의 사진을 삭제한다.
    camera.stop_preview()
    camera.close()

#메모리 카드의 파일을 정리 해 주자.
def clearAll():
    #제대로 할려면 용량 체크 하고 먼저 촬영된 이미지 부터 지워야 할것 같지만 여기선 폴더안에 파일을 몽땅 지우자.
    path = 'captureImages'
    os.system('rm -rf %s/*' % path)

fileUpload("bridge.jpeg")

# #30초 마다 실행
# #schedule.every(10).seconds.do(execute_camera)
# #10분에 한번씩 실행
# schedule.every(10).minutes.do(execute_camera)
# #매 시간 마다 실행
# schedule.every().hour.do(clearAll)
# #기타 정해진 시간에 실행/매주 월요일에 실행/매주 수요일 몇시에 실행 등의 옵션이 있다.


# while True:
#     schedule.run_pending()
#     sleep(1)

ModuleNotFoundError: ignored

In [ ]:
!apt-get install python-picamera

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package python-picamera
